<a href="https://colab.research.google.com/github/AyrtonPinheiro/Aprendizado_Maquina-Tarefa04/blob/main/Aprendizado_Maquinas_trabalho_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
try:
    import google.colab
    import requests
    url = 'https://raw.githubusercontent.com/dvgodoy/PyTorchStepByStep/master/config.py'
    r = requests.get(url, allow_redirects=True)
    open('config.py', 'wb').write(r.content)
except ModuleNotFoundError:
    pass

from config import *
config_chapter7()
# This is needed to render the plots in this chapter
from plots.chapter7 import *

Finished!


In [ ]:
from stepbystep.v3 import StepByStep

In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
from torchvision.transforms.v2 import Compose, ToImage, Normalize,  \
Resize, ToPILImage, CenterCrop, RandomResizedCrop, ToDtype, ToTensor
from torchvision.datasets import ImageFolder
from torchvision.models import alexnet, resnet18, inception_v3

# Updated for Torchvision 0.15
from torchvision.models.alexnet import AlexNet_Weights
from torchvision.models.inception import Inception_V3_Weights
from torchvision.models.resnet import ResNet18_Weights
# from torchvision.models.alexnet import model_urls

import os
import shutil

# Dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gpiosenka/cards-image-datasetclassification")

print("Path to dataset files:", path)

100%|██████████| 385M/385M [00:03<00:00, 117MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/gpiosenka/cards-image-datasetclassification/versions/2


In [ ]:
# Create the destination folder if it doesn't exist
rps_folder = os.path.join("/content/", "rps")
os.makedirs(rps_folder, exist_ok=True)

# Iterate through the subfolders in the train folder
train_folder = os.path.join(path, "train")

for folder_name in os.listdir(train_folder):
    source_folder = os.path.join(train_folder, folder_name)
    destination_folder = os.path.join(rps_folder, folder_name)

    # Check if it's a directory before moving
    if os.path.isdir(source_folder):
        shutil.move(source_folder, destination_folder)

In [ ]:
rps_folder = os.path.join("/content/", "rps-test-set")
os.makedirs(rps_folder, exist_ok=True)

# Iterate through the subfolders in the train folder
train_folder = os.path.join(path, "test")

for folder_name in os.listdir(train_folder):
    source_folder = os.path.join(train_folder, folder_name)
    destination_folder = os.path.join(rps_folder, folder_name)

    # Check if it's a directory before moving
    if os.path.isdir(source_folder):
        shutil.move(source_folder, destination_folder)

In [ ]:
# ImageNet statistics
normalizer = Normalize( mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])

composer = Compose([Resize(224),
                    CenterCrop(224),
                    ToImage(),
                    ToDtype(torch.float32, scale=True),
                    normalizer])

train_data = ImageFolder(root='rps', transform=composer)
val_data = ImageFolder(root='rps-test-set', transform=composer)

# Builds a loader of each set
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16)

# Model Finetuning

In [ ]:
# repeating import from the top for reference
from torchvision.models.resnet import ResNet18_Weights

# UPDATED
###########################################################
# This is the recommended way of loading a pretrained
# model's weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
# model = resnet18(pretrained=True)
###########################################################

torch.manual_seed(42)
model.fc = nn.Linear(512, 53)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 188MB/s]


In [ ]:
multi_loss_fn = nn.CrossEntropyLoss(reduction='mean')
optimizer_model = optim.Adam(model.parameters(), lr=3e-4)
sbs_transfer = StepByStep(model, multi_loss_fn, optimizer_model)

In [ ]:
sbs_transfer.set_loaders(train_loader, val_loader)
sbs_transfer.train(1)

In [ ]:
StepByStep.loader_apply(val_loader, sbs_transfer.correct)

tensor([[5, 5],
        [5, 5],
        [5, 5],
        [2, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [4, 5],
        [4, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [4, 5],
        [5, 5],
        [3, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [4, 5],
        [5, 5],
        [4, 5],
        [5, 5],
        [3, 5],
        [3, 5],
        [5, 5],
        [4, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [4, 5],
        [4, 5],
        [4, 5],
        [5, 5],
        [4, 5],
        [5, 5],
        [4, 5],
        [4, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [5, 5]])

# Model Feature Extractor

In [ ]:
def freeze_model(model):
  for parameter in model.parameters():
    parameter.requires_grad = False

In [ ]:
# repeating import from the top for reference
from torchvision.models.resnet import ResNet18_Weights

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# UPDATED
###########################################################
model = resnet18(weights=ResNet18_Weights.DEFAULT).to(device)
# model = resnet18(pretrained=True).to(device)
###########################################################

model.fc = nn.Identity()
freeze_model(model)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 186MB/s]


In [ ]:
def preprocessed_dataset(model, loader, device=None):
  if device is None:
    device = next(model.parameters()).device

  features = None
  labels = None

  for i, (x, y) in enumerate(loader):
    model.eval()
    output = model(x.to(device))
    if i == 0:
      features = output.detach().cpu()
      labels = y.cpu()
    else:
      features = torch.cat([features, output.detach().cpu()])
      labels = torch.cat([labels, y.cpu()])

  dataset = TensorDataset(features, labels)
  return dataset

In [ ]:
train_preproc = preprocessed_dataset(model, train_loader, device)
val_preproc = preprocessed_dataset(model, val_loader, device)
train_preproc_loader = DataLoader(train_preproc, batch_size=16, shuffle=True)
val_preproc_loader = DataLoader(val_preproc, batch_size=16)

In [ ]:
torch.manual_seed(42)
top_model = nn.Sequential(nn.Linear(512, 53))
multi_loss_fn = nn.CrossEntropyLoss(reduction='mean')
optimizer_top = optim.Adam(top_model.parameters(), lr=3e-4)

In [ ]:
sbs_top = StepByStep(top_model, multi_loss_fn, optimizer_top)
sbs_top.set_loaders(train_preproc_loader, val_preproc_loader)
sbs_top.train(100)

In [ ]:
StepByStep.loader_apply(val_preproc_loader, sbs_top.correct)

tensor([[4, 5],
        [4, 5],
        [5, 5],
        [5, 5],
        [4, 5],
        [1, 5],
        [4, 5],
        [3, 5],
        [2, 5],
        [2, 5],
        [2, 5],
        [2, 5],
        [2, 5],
        [4, 5],
        [2, 5],
        [2, 5],
        [4, 5],
        [1, 5],
        [2, 5],
        [2, 5],
        [0, 5],
        [5, 5],
        [4, 5],
        [1, 5],
        [2, 5],
        [2, 5],
        [1, 5],
        [3, 5],
        [4, 5],
        [3, 5],
        [0, 5],
        [3, 5],
        [4, 5],
        [2, 5],
        [1, 5],
        [2, 5],
        [3, 5],
        [1, 5],
        [2, 5],
        [1, 5],
        [2, 5],
        [3, 5],
        [2, 5],
        [3, 5],
        [3, 5],
        [2, 5],
        [5, 5],
        [3, 5],
        [3, 5],
        [2, 5],
        [2, 5],
        [1, 5],
        [3, 5]])

In [ ]:
model.fc = top_model
sbs_temp = StepByStep(model, None, None)

In [ ]:
StepByStep.loader_apply(val_loader, sbs_temp.correct)


tensor([[4, 5],
        [4, 5],
        [5, 5],
        [5, 5],
        [4, 5],
        [1, 5],
        [4, 5],
        [3, 5],
        [2, 5],
        [2, 5],
        [2, 5],
        [2, 5],
        [2, 5],
        [4, 5],
        [2, 5],
        [2, 5],
        [4, 5],
        [1, 5],
        [2, 5],
        [2, 5],
        [0, 5],
        [5, 5],
        [4, 5],
        [1, 5],
        [2, 5],
        [2, 5],
        [1, 5],
        [3, 5],
        [4, 5],
        [3, 5],
        [0, 5],
        [3, 5],
        [4, 5],
        [2, 5],
        [1, 5],
        [2, 5],
        [3, 5],
        [1, 5],
        [2, 5],
        [1, 5],
        [2, 5],
        [3, 5],
        [2, 5],
        [3, 5],
        [3, 5],
        [2, 5],
        [5, 5],
        [3, 5],
        [3, 5],
        [2, 5],
        [2, 5],
        [1, 5],
        [3, 5]])